In [1]:
cd ..

/Users/jwen/Projects/road_building_extraction


In [2]:
cd src

/Users/jwen/Projects/road_building_extraction/src


In [3]:
import sys
import warnings
warnings.simplefilter("ignore", (UserWarning, FutureWarning))

from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from skimage import io
from PIL import Image
from tqdm import tqdm
from torchvision import transforms

import pandas as pd
import tensorflow as tf
import glob
import numpy as np
import random
import torch

from utils import data_utils
from utils import augmentation as aug

%matplotlib inline

# Data directory structure
We have pulled the data that we need, but the structure of the data will be difficult to deal with when we are training and testing. Create a .csv with the file paths to the images and the subdirectories

In [107]:
# one csv for each of the main datasets
mass_buildings = glob.glob('/Users/jwen/Projects/road_building_extraction/data/mass_buildings/**/*.tiff', recursive=True)
mass_roads = glob.glob('/Users/jwen/Projects/road_building_extraction/data/mass_roads/**/*.tiff', recursive=True)
mass_roads_crop = glob.glob('/Users/jwen/Projects/road_building_extraction/data/mass_roads_crop/**/*.tiff', recursive=True)

mass_buildings_df = pd.DataFrame(mass_buildings)
mass_roads_df = pd.DataFrame(mass_roads)
mass_roads_crop_df = pd.DataFrame(mass_roads_crop)

mass_buildings_df.rename(columns={0:'file_path'}, inplace=True)
mass_roads_df.rename(columns={0:'file_path'}, inplace=True)
mass_roads_crop_df.rename(columns={0:'file_path'}, inplace=True)

In [108]:
# create new column with the split 
mass_buildings_df['sat_img_path'] = mass_buildings_df['file_path'].apply(lambda x: x.split('/')[-1])
mass_buildings_df['map_img_path'] = mass_buildings_df['file_path'].apply(lambda x: x.split('/')[-1][:-1])
mass_buildings_df['sat_map'] = mass_buildings_df['file_path'].apply(lambda x: x.split('/')[-2])
mass_buildings_df['train_valid_test'] = mass_buildings_df['file_path'].apply(lambda x: x.split('/')[-3])

mass_roads_df['sat_img_path'] = mass_roads_df['file_path'].apply(lambda x: x.split('/')[-1])
mass_roads_df['map_img_path'] = mass_roads_df['file_path'].apply(lambda x: x.split('/')[-1][:-1])
mass_roads_df['sat_map'] = mass_roads_df['file_path'].apply(lambda x: x.split('/')[-2])
mass_roads_df['train_valid_test'] = mass_roads_df['file_path'].apply(lambda x: x.split('/')[-3])

mass_roads_crop_df['sat_img_path'] = mass_roads_crop_df['file_path'].apply(lambda x: x.split('/')[-1])
mass_roads_crop_df['map_img_path'] = mass_roads_crop_df['file_path'].apply(lambda x: x.split('/')[-1][:-1])
mass_roads_crop_df['sat_map'] = mass_roads_crop_df['file_path'].apply(lambda x: x.split('/')[-2])
mass_roads_crop_df['train_valid_test'] = mass_roads_crop_df['file_path'].apply(lambda x: x.split('/')[-3])

In [112]:
# create csv file with the files paths to the data
# mass_buildings_df.to_csv('/Users/jwen/Projects/road_building_extraction/data/mass_buildings/mass_buildings.csv', index=False)
# mass_roads_df.to_csv('/Users/jwen/Projects/road_building_extraction/data/mass_roads/mass_roads.csv', index=False)
# mass_roads_crop_df.to_csv('/Users/jwen/Projects/road_building_extraction/data/mass_roads_crop/mass_roads_crop.csv', index=False)

In [113]:
# total observations for new cropped dataset
len(mass_roads_crop_df)

13860

In [114]:
mass_roads_crop_df.head()

,file_path,sat_img_path,map_img_path,sat_map,train_valid_test
0,/Users/jwen/Projects/road_building_extraction/...,10_10228690_15.tiff,10_10228690_15.tif,sat,valid
1,/Users/jwen/Projects/road_building_extraction/...,12_24929245_15.tiff,12_24929245_15.tif,sat,valid
2,/Users/jwen/Projects/road_building_extraction/...,0_23128930_15.tiff,0_23128930_15.tif,sat,valid
3,/Users/jwen/Projects/road_building_extraction/...,4_10228690_15.tiff,4_10228690_15.tif,sat,valid
4,/Users/jwen/Projects/road_building_extraction/...,6_24929245_15.tiff,6_24929245_15.tif,sat,valid


# Testing Data Loading

In [7]:
# load data with transformatons
# mass_dataset_train = MassRoadBuildingDataset('/Users/jwen/Projects/road_building_extraction/data/mass_roads/mass_roads.csv','mass_roads','train',
#                                        transform=transforms.Compose([RescaleTarget(268), RandomCropTarget(238), ToTensorTarget()]))


# mass_dataset_val = MassRoadBuildingDataset('/Users/jwen/Projects/road_building_extraction/data/mass_roads/mass_roads.csv','mass_roads','valid',
#                                        transform=transforms.Compose([RescaleTarget(268), ToTensorTarget()]))

data_path = '/Users/jwen/Projects/road_building_extraction/data/mass_roads_crop/mass_roads_crop.csv'
data_set = 'mass_roads_crop'

mass_dataset_train = data_utils.MassRoadBuildingDataset(data_path, data_set, 'train',transform=transforms.Compose([aug.ToTensorTarget()]))

mass_dataset_val = data_utils.MassRoadBuildingDataset(data_path, data_set, 'valid')
    
# RandomRotationTarget(45,resize=True), NormalizeTarget(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

In [8]:
dataloader = DataLoader(mass_dataset_train, batch_size=3, num_workers=4)
data_batch = next(iter(dataloader))

dataloader_valid = DataLoader(mass_dataset_val, batch_size=6, num_workers=4)
data_batch_valid = next(iter(dataloader_valid))

In [184]:
transforms.functional.to_tensor(mass_dataset_train[0]['sat_img'])


( 0 ,.,.) = 
  0.2000  0.1216  0.1882  ...   0.5882  0.5647  0.5451
  0.1490  0.1059  0.0980  ...   0.6431  0.6314  0.6196
  0.0980  0.0784  0.0980  ...   0.6275  0.6549  0.6471
           ...             ⋱             ...          
  0.6980  0.5098  0.5216  ...   0.4078  0.4157  0.4039
  0.4902  0.5608  0.4863  ...   0.4157  0.4000  0.3922
  0.5020  0.4980  0.4549  ...   0.4196  0.3922  0.3804

( 1 ,.,.) = 
  0.2392  0.1373  0.1882  ...   0.5804  0.5608  0.5451
  0.1059  0.0863  0.0980  ...   0.6392  0.6235  0.6118
  0.0980  0.0980  0.1137  ...   0.6235  0.6431  0.6431
           ...             ⋱             ...          
  0.7020  0.5098  0.5333  ...   0.4275  0.4314  0.4353
  0.4980  0.5569  0.4863  ...   0.4314  0.4039  0.3843
  0.5137  0.5137  0.4745  ...   0.4353  0.3922  0.3922

( 2 ,.,.) = 
  0.1765  0.1059  0.1294  ...   0.5608  0.5373  0.5255
  0.0980  0.0706  0.0667  ...   0.6118  0.5922  0.5765
  0.0549  0.0745  0.0824  ...   0.5882  0.6000  0.6039
           ...         